# Plot hovmøller diagram of leads along transect in Beaufort Sea

In [ ]:
# allow plots to be interactive in the notebook
%matplotlib notebook

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import os 
import sys
import xarray as xr
import matplotlib.colors as colors
import cartopy 
import cartopy.crs as ccrs
import pyproj
from pynextsim.projection_info import ProjectionInfo
import matplotlib.gridspec as gridspec
import cmocean

In [ ]:
# Load data

k=0 #what exp to plot
var='hfs'
exp = ['expt_00_wrf10km', 'expt_05_wrf_40km', 'expt_02_era5',  'expt_03_cfsr'] 
rootdir = '/cluster/work/users/rheinlender/breakup2013/WRF-exp/outputs/from_Tim/'+exp[k]+'/outputs/'
filename='Moorings.nc'


exp = ['wrf10'] 
rootdir = '/cluster/work/users/rheinlender/breakup2013/WRF-exp/outputs/'+exp[k]+'/'

fl=rootdir+filename
print(fl)
    
# use xarray to open file
nc = xr.open_dataset(fl); 
nc

In [ ]:
# Plot transect in Beaufort Sea

# Create slice variables subset domain
start_time = '2013-02-11 06:00'
end_time = '2013-03-08 18:00'
time_slice = slice(start_time, end_time)
#x_slice = 50
#y_slice = slice(90, 230)
x_slice = 100
y_slice = slice(190, 400)

# get transect
transect = nc[var].sel(time=time_slice, x=x_slice, y=y_slice)
thickness = nc['sit'].sel(time=time_slice, x=x_slice, y=y_slice)
concentration = nc['sic'].sel(time=time_slice, x=x_slice, y=y_slice)

# Get times and make array of datetime objects
vtimes = nc['time'].sel(time=time_slice).values.astype('datetime64[ms]').astype('O')

# Specify y values for transect
length = transect.sizes['y']
yvals = np.linspace(y_slice.start, y_slice.stop,length) 

# PLOTTING
plt.close('all')
fig = plt.figure(figsize=(6, 4))

# Use gridspec to help size elements of plot; small top plot and big bottom plot
gs = gridspec.GridSpec(nrows=2, ncols=2, width_ratios=[2, 6], hspace=0.03)

ax1 = fig.add_subplot(gs[0, 0])
nc[var][0,:,:].plot.imshow(ax=ax1, cmap='rainbow', vmin=0, vmax=200, add_colorbar=False)
ax1.set_title("")
# add line for transect
ax1.plot([x_slice, x_slice], [y_slice.start, y_slice.stop], 'ro-')

# Bottom plot for Hovmoller diagram
ax2 = fig.add_subplot(gs[0:, 1:])
clevs = np.arange(0, 200+10, 10)
clevs2 = [0.6]
cf = ax2.contourf(vtimes, yvals, transect.transpose(), clevs, cmap='rainbow', extend='max')
cs = ax2.contour(vtimes, yvals, thickness.transpose(), clevs2, colors='black', linewidths=1)

cbar = plt.colorbar(cf, orientation='vertical', pad=0.04, aspect=50, extend='max')
cbar.set_label('$\mathrm{W \: m^{-2}}$')

# rotate xlabels
plt.xticks(rotation=45)

ax2.set_ylabel("y")
ax2.set_title(exp[k])

# save figure
outpath_plots = '/cluster/home/rheinlender/projects/aoi_case_study/python/plots/wrf/'
figname = outpath_plots + var + 'Transect_20130211-20130308_' + exp[k] + '.png'
fig.savefig(figname, dpi=150, bbox_inches='tight')